https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
# vit_b_16
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        # transforms.InterpolationMode('bilinear'),
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        # transforms.InterpolationMode('bilinear'),
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [8]:
data_dir = 'E:\Datasets\cor'
data_folders = ['train', 'val']
batch_size = 64
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
trainloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          # num_workers=0
                                          )
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
testloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          # num_workers=0
                                          )

In [ ]:
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
# curr_time = datetime(curr_time.year, curr_time.month, curr_time.day, curr_time.hour)
curr_time = curr_time.strftime("%m/%d/%Y %H:%M:%S")

output_folder = os.path.join(models_folder, curr_time)
if not os.path.exists(output_folder): 
    os.makedirs(output_folder) 

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = models.vit_b_16(pretrained=True)
model.load_state_dict(torch.load('/notebooks/vit/models/10-27-2023 10-39-19/epoch_4.pth'))
model.to(device) 

c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

criterion = nn.CrossEntropyLoss()   
optimizer = optim.SGD(model.heads.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 25
model_epoch_interval = 2
log = open(f"{output_folder}/log.txt",'w')
log.close()

for epoch in range(1,num_epochs+1):  # loop over the dataset multiple times
    running_loss = 0.0
    
    epoch_info = f'Epoch {epoch}/{num_epochs}'
    with open(f"{output_folder}/log.txt",'a') as log:
                log.write('\n'+'-'*10+epoch_info+'\n'+'-'*10+'\n')
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            text = f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}'
            print(text)
            with open(f"{output_folder}/log.txt",'a') as log:
                log.write(text+'\n')
            running_loss = 0.0
    
    if epoch % model_epoch_interval == 0:
        torch.save(model.state_dict, f'{output_folder}/epoch_{epoch}.pth')

torch.save(model.state_dict, f'{output_folder}/last.pth')
print('Finished Training')

In [ ]:
# #### ConvNet as fixed feature extractor ####
# # Here, we need to freeze all the network except the final layer.
# # We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
# model_conv = models.resnext101_32x8d(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)

# model_conv = model_conv.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# model_conv = train_model(model_conv, criterion, optimizer_conv,
#                          exp_lr_scheduler, num_epochs=25)